In [ ]:
import torch
import numpy as np
import imageio
import pickle
import matplotlib.pyplot as plt

from lucent.optvis import render, param, objectives
from lucent.modelzoo import inceptionv1
from lucent.modelzoo.util import get_model_layers
from lucent.optvis.render import pixelwise_gradient_change_interrupt


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

param_f = lambda: param.image(224, batch=9)
opt = lambda params: torch.optim.Adam(params, 0.05)


In [ ]:
#getting googlenet
 
model = inceptionv1(pretrained=True)
model.to(device).eval()

layer = "mixed3a_1x1_pre_relu_conv" # "mixed5b_pool_reduce_pre_relu_conv" #  "mixed3a_1x1_pre_relu_conv" # "mixed5b_pool_reduce_pre_relu_conv" # 
channel = 34 # 63 # 34
diversity_alpha = 100 # 6062.5 # super large objective which should hinder optimization

In [ ]:
# alternatively, getting resnet50

from resnet import get_resnet
model = get_resnet("standard", device) 

layer = "layer3_1_conv3" #"layer4_1_conv3" # "layer3_1_conv3"
channel = 42
diversity_alpha = 100

In [ ]:
obj = objectives.channel(layer, channel)
# obj -= diversity_alpha * objectives.diversity(layer)

In [ ]:
# seems to work well for both model types

# to get the gradients, uncomment the return statement in render.py

images, raw_grads = render.render_vis(model, obj, param_f, opt, thresholds=(20000,), verbose=True, layer=layer, channel=channel, min_steps=2500, interrupt_interval=1, interrupt_condition=pixelwise_gradient_change_interrupt)

In [ ]:
gradfile = f"{layer}__{channel}__grads_l2_resnet.pkl"

with open(gradfile, "wb") as f:
    pickle.dump(raw_grads, f)

In [ ]:
#layer = "mixed5b_pool_reduce_pre_relu_conv" #  "mixed3a_1x1_pre_relu_conv" # "mixed5b_pool_reduce_pre_relu_conv" # 
#channel = 63 # 34
#diversity_alpha = 6062.5 # super large objective which should hinder optimization

filename = f"{layer}__{channel}__grads_l2_resnet.pkl"

with open(filename, "rb") as f:
    grads = pickle.load(f)


def ratio(sequence):
    winsize = 2000

    # continue if we haven't even collected enough values
    if len(sequence) < winsize:
        return 2

    first_half = np.mean(sequence[-winsize:-winsize//2])
    second_half = np.mean(sequence[-winsize//2:])

    ratio = first_half / second_half

    return ratio


def smooth(scalars, weight: float):  # Weight between 0 and 1
    last = scalars[0]  # First value in the plot (first timestep)
    smoothed = list()
    for point in scalars:
        smoothed_val = last * weight + (1 - weight) * point  # Calculate smoothed value
        smoothed.append(smoothed_val)                        # Save it
        last = smoothed_val                                  # Anchor the last smoothed value

    return smoothed


def plot_grads(grads):
    xs = list(range(len(grads)))
    ratios = [ratio(grads[:i]) for i in range(len(grads))]
    #smoothed = smooth(grads, 0.99) #[np.mean(grads[idx-win:idx+win]) for idx in range(win, len(acts)-win)]

    start = 500
    xs = xs[start:]
    grads = grads[start:]
    ratios = ratios[start:]
    #smoothed = smoothed[start:]

    fig, ax1 = plt.subplots()
    plt.title(f"Mean pixelwise grads for {layer}__{channel}, alpha {diversity_alpha}")

    ax1.set_xlabel("Optimization Steps")
    ax1.set_ylabel("Mean pixelwise grad")
    #ax1.set_yscale('log')
    # ax1.set_ylim(0, 30)

    ax1.plot(xs, grads)
    #ax1.plot(xs, smoothed)
    #ax1.plot(xs, [8]*len(xs), c='r')

    ax2 = ax1.twinx()
    ax2.set_ylabel("Ratio between windows")
    ax2.plot(xs, ratios, c='g')
    ax2.set_ylim(1, 1.5)
    ax2.plot(xs, [1.1]*len(xs), c='r')

    plt.show()
    #plt.savefig(f"activations_{layer}_{channel}.png")
    plt.close()

plot_grads(grads)